# Loading the data

In [120]:
import pandas as pd
import os
print(os.getcwd())
df_source = pd.read_csv("data/transcripts/whole_data_2021_2016.csv")
df_source_tot = pd.read_csv("data/transcripts/2016_full_behaviors_annotation_w_hedges.csv")

/Users/ewenmichel/Desktop/Centrale/3A/SDI/Project/Multimodal-CSC


In [121]:
columns = pd.read_csv("data/transcripts/clean_D3_S1.csv").columns
df_time = pd.DataFrame(columns = pd.read_csv("data/transcripts/clean_D3_S1.csv").columns)

for file in os.listdir("./data/transcripts/"):
    if file.startswith("clean_"):
        curr_df = pd.read_csv('./data/transcripts/' + file)
        if "Temps de départ - hh:mm:ss.ms" in curr_df.columns:
            curr_df["Begin Time - hh:mm:ss.ms"] = curr_df["Temps de départ - hh:mm:ss.ms"]
            curr_df = curr_df.drop(columns = ["Temps de départ - hh:mm:ss.ms"])
        if 'temps de fin - hh:mm:ss.ms' in curr_df.columns:
            curr_df['End Time - hh:mm:ss.ms'] = curr_df['temps de fin - hh:mm:ss.ms']
            curr_df = curr_df.drop(columns = ['temps de fin - hh:mm:ss.ms'])
        if 'Durée - hh:mm:ss.ms' in curr_df.columns:
            curr_df['Duration - hh:mm:ss.ms'] = curr_df['Durée - hh:mm:ss.ms']
            curr_df = curr_df.drop(columns = ['Durée - hh:mm:ss.ms'])
        df_time = pd.concat([df_time, curr_df])

        if '4_S2' in file:
            print(curr_df.columns)

df_time = df_time.drop(columns = ["Unnamed: 0"])
df_time = df_time[['Dyad', 'Session', 'Begin Time - hh:mm:ss.ms', 'End Time - hh:mm:ss.ms', 'Duration - hh:mm:ss.ms', 'P1', 'P2']]
df_time['Time_begin_ts'] = pd.to_datetime(df_time["Begin Time - hh:mm:ss.ms"], format = "%H:%M:%S.%f")

print(len(df_time))

Index(['Unnamed: 0', 'Dyad', 'Session', 'P1', 'P2', 'Begin Time - hh:mm:ss.ms',
       'End Time - hh:mm:ss.ms', 'Duration - hh:mm:ss.ms'],
      dtype='object')
11133


In [122]:
df_source["Time_begin_ts"] = pd.to_datetime(df_source["Time_begin"], format = "%M:%S.%f")
df_time['Time_begin_ts'] = pd.to_datetime(df_time["Begin Time - hh:mm:ss.ms"], format = "%H:%M:%S.%f")

In [123]:
print(df_time["Time_begin_ts"].iloc[0] + pd.Timedelta(seconds = 10) < df_time["Time_begin_ts"].iloc[0])

False


In [140]:
# df_source = pd.read_csv("data/transcripts/whole_data_2021_2016.csv")
df_source = pd.read_csv("data/transcripts/2016_full_behaviors_annotation_w_hedges.csv")
for col_ in ["Time_begin", "Time_end"]:
    df_source.loc[:, col_] = df_source.loc[:, col_].apply(lambda x : int(len(str(x)) == len("00:01:25")) * (str(x) + ".000000") + int(len(str(x)) != 8) * str(x))

mask = ((df_source.Dyad == 6) & (df_source.Session == 1))

# df_source.loc[mask, ["P1", "P2"]] = pd.concat([df_source.loc[mask].P2, df_source.loc[mask].P1], axis = 1)
df_source.loc[mask, ["P1", "P2"]] = df_source.loc[mask, ["P2", "P1"]].values

df_source["Time_begin_ts"] = pd.to_datetime(df_source["Time_begin"], format = "%H:%M:%S.%f")
df_source = df_source.sort_values(by = ["Dyad", "Session", "Time_begin"])

sd_df = df_source[(df_source["SD_Tutor"] == "SD") | (df_source["SD_Tutee"] == "SD")].replace("SD", "x").fillna("")
qe_df = df_source[(df_source["SD_Tutor"] == "QE") | (df_source["SD_Tutee"] == "QE")].replace("QE", "x").fillna("")
sv_df = df_source[(df_source["SV_Tutor"] == "SV") | (df_source["SV_Tutee"] == "SV")].replace("SV", "x").fillna("")
pr_df = df_source[(df_source["PR_Tutor"].isin(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP'])) | (df_source["PR_Tutee"].isin(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP']))].replace(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP'], "x").fillna("")
hd_df = df_source[(df_source["HD_Tutor"].isin(['IDQ/IDS', 'IDQ', 'IDS', 'IDE', 'IDA', 'IDA/IDQ', 'IDQ/IDE', 'IDE/IDS'])) | (df_source["HD_Tutee"].isin(['IDQ/IDS', 'IDQ', 'IDS', 'IDE', 'IDA', 'IDA/IDQ', 'IDQ/IDE', 'IDE/IDS']))].replace(['IDQ/IDS', 'IDQ', 'IDS', 'IDE', 'IDA', 'IDA/IDQ', 'IDQ/IDE', 'IDE/IDS'], "x").fillna("")

sd_df["SD"] = sd_df.SD_Tutor + sd_df.SD_Tutee
qe_df["QE"] = qe_df.SD_Tutor + qe_df.SD_Tutee
sv_df["SV"] = sv_df.SV_Tutor + sv_df.SV_Tutee
pr_df["PR"] = pr_df.PR_Tutor + pr_df.PR_Tutee
hd_df["HD"] = hd_df.HD_Tutor + hd_df.HD_Tutee

sd_map_dict = dict(zip(sd_df.index, sd_df["SD"]))
qe_map_dict = dict(zip(qe_df.index, qe_df["QE"]))
sv_map_dict = dict(zip(sv_df.index, sv_df["SV"]))
pr_map_dict = dict(zip(pr_df.index, pr_df["PR"]))
hd_map_dict = dict(zip(hd_df.index, hd_df["HD"]))

df_source["SD"] = df_source.index.to_series().map(sd_map_dict)
df_source["QE"] = df_source.index.to_series().map(qe_map_dict)
df_source["SV"] = df_source.index.to_series().map(sv_map_dict)
df_source["PR"] = df_source.index.to_series().map(pr_map_dict)
df_source["HD"] = df_source.index.to_series().map(hd_map_dict)

df_source = df_source.drop(df_source[df_source["SV"] == "xx"].index)
for l_ in ["SD", "PR", "QE", "SV", "HD"]:
    print(len(df_source[df_source[l_] == 'x']))
df_source["SD"].unique(), df_source["QE"].unique(), df_source["SV"].unique(), df_source["PR"].unique(), df_source["HD"].unique()

738
114
163
2240
1368


(array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object))

In [141]:
mask = (df_source.Dyad == 6) & (df_source.Session == 1) & (df_source.P1.notna())
df_source.loc[mask, "P1"].iloc[40:]

13799     the last study I did here took like five minutes
13800     and they paid me twenty bucks an hour and a half
13801    I didn't really know I was signing up to do ma...
13803                                         yeah are you
13805                            where do you go to school
                               ...                        
14554                                              goodbye
14556    I'll probably see you at some TechGirls soon-i...
14558    I think my mom is forcing me to come on Monday so
14559                     pause filler you can hang up now
14560                                                 okay
Name: P1, Length: 504, dtype: object

In [142]:
df_source = df_source[['Time_begin', 'Dyad', 'Session',  'P1', 'P2', 'SD', 'QE', 'SV', 'PR', 'HD']]

In [143]:
df_time = df_time[['Dyad', 'Session', 'Begin Time - hh:mm:ss.ms', 'End Time - hh:mm:ss.ms', 'Duration - hh:mm:ss.ms', 'P1', 'P2', 'Time_begin_ts']]

In [144]:
df_time["Begin Time - hh:mm:ss.ms"].iloc[0]

'00:00:00.250'

In [145]:
df_source["Time_begin"].iloc[0]

'00:00:00.100000'

In [146]:
df_result = df_time.merge(df_source, on = ['Dyad', 'Session', 'P1', 'P2'], how = "left")
df_result = df_result.drop_duplicates().sort_values(by = ['Dyad', 'Session',  'Begin Time - hh:mm:ss.ms'])
# df_result = df_result.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]

In [147]:
    # if dyad_ == 11: 
    #     df_source_fz_ = df_source[(df_source["Dyad"] == dyad_) & (df_source["Session"] == 1)]
    #     df_time_fz_ = df_time[(df_time["Dyad"] == dyad_) & (df_time["Session"] == 1)]

    #     df_source_fz_ = df_source_fz_[(df_source_fz_[f"P1"].notna()) | (df_source_fz_[f"P2"].notna())]
    #     df_time_fz_ = df_time_fz_[(df_time_fz_[f"P1"].notna()) | (df_time_fz_[f"P2"].notna())]

    #     df_source_fz_["P1"].loc[df_source_fz_["P1"].notna()] = "1" + df_source_fz_["P1"]
    #     df_source_fz_["P2"].loc[df_source_fz_["P2"].notna()] = "2" + df_source_fz_["P2"]

    #     df_source_fz_ = df_source_fz_.fillna('')
    #     df_source_fz_["P"] = df_source_fz_["P1"] + df_source_fz_["P2"]

    #     source_P = df_source_fz_["P"].tolist()
    #     source_timestamps = df_source_fz_["Time_begin"] # mm:ss.ms

    #     for utt_, ts_ in zip(source_P, source_timestamps):
    #         ts_ = pd.to_datetime(ts_, format = "%M:%S.%f")
    #         curr_df_time_fz_ = df_time_fz_[(df_time_fz_["Time_begin_ts"] < ts_ + pd.Timedelta(seconds=10)) | (df_time_fz_["Time_begin_ts"] > ts_ - pd.Timedelta(seconds=10))]
    #         time_P1 = curr_df_time_fz_["P1"].tolist()
    #         time_P2 = curr_df_time_fz_["P2"].tolist()
    #         if utt_[0] == "1":
    #             result.append(process.extract(utt_, time_P1, limit = 2))
    #         elif utt_[0] == "2":
    #             result.append(process.extract(utt_, time_P2, limit = 2))

In [160]:
from fuzzywuzzy import fuzz, process

df_source = df_source[(df_source.SD == "x") | (df_source.PR == "x") | (df_source.HD == "x") | (df_source.QE == "x") | (df_source.SV == "x")]

result = []

dyad_list = [3,4,5,6,7,8,10]
session_list = [1,2]
# person_list = [1, 2]
threshold = 20

df_source = df_source[(df_source["P1"].notna()) ^ (df_source["P2"].notna())]

for dyad_ in dyad_list:
    for session_ in session_list:
        df_source_fz_ = df_source[(df_source["Dyad"] == dyad_) & (df_source["Session"] == session_)]
        df_time_fz_ = df_time[(df_time["Dyad"] == dyad_) & (df_time["Session"] == session_)]

        df_source_fz_ = df_source_fz_[(df_source_fz_[f"P1"].notna()) | (df_source_fz_[f"P2"].notna())]
        df_time_fz_ = df_time_fz_[(df_time_fz_[f"P1"].notna()) | (df_time_fz_[f"P2"].notna())]

        df_source_fz_.loc[df_source_fz_["P1"].notna(), "P1"] = "1" + df_source_fz_["P1"]
        df_source_fz_.loc[df_source_fz_["P2"].notna(), "P2"] = "2" + df_source_fz_["P2"]

        df_source_fz_ = df_source_fz_.fillna('')
        df_source_fz_["P"] = df_source_fz_["P1"] + df_source_fz_["P2"]

        source_P = df_source_fz_["P"].tolist()
        source_timestamps = df_source_fz_["Time_begin"] # mm:ss.ms

        for utt_, ts_ in zip(source_P, source_timestamps):
            ts_ = pd.to_datetime(ts_, format = "%H:%M:%S.%f")
            utt_ = utt_.replace("pause filler", "").replace("(laughter)", "")
            curr_df_time_fz_ = df_time_fz_[(df_time_fz_["Time_begin_ts"] < ts_ + pd.Timedelta(seconds=45)) & (df_time_fz_["Time_begin_ts"] > ts_ - pd.Timedelta(seconds=45))]
            time_P1 = curr_df_time_fz_["P1"].apply(lambda x : int(str(x) != "nan") * (str(x).strip("\""))).tolist()
            time_P2 = curr_df_time_fz_["P2"].apply(lambda x : int(str(x) != "nan") * (str(x).strip("\""))).tolist()
            if utt_[0] == "1":
                # if "I used to play piano" in time_P1:
                #     print(utt_)
                #     print(process.extract(utt_[1:], time_P1))
                match = process.extract(utt_[1:], time_P1)
                if False:
                    match = match[0]
                result.append(match)
            elif utt_[0] == "2":
                match = process.extract(utt_[1:], time_P2)
                if False: # match[0][1]==100
                    match = match[0]
                result.append(match)

            # df_source.loc[((df_source["Dyad"] == dyad_) & (df_source["Session"] == session_) & (df_source["P1"].notna()))]['match'] = result

df_source_result = df_source[((df_source["P1"].notna()) ^ (df_source["P2"].notna())) & (df_source["Dyad"].isin(dyad_list)) & (df_source["Session"].isin(session_list))]
df_source_result["Match"] = result
# df_source_fz = df_source[df_source["P1"].notna()]
# df_time_fz = df_source[df_source["P1"].notna()]

# source_P1 = df_source_fz["P1"].tolist()
# time_P1 = df_time_fz["P1"].tolist()

# threshold = 60

# result = []

# for utt in source_P1:
#     result.append(process.extract(utt, time_P1, limit = 3))

/var/folders/l7/1yhnrkcx2f79z5dph4rg3h540000gn/T/ipykernel_63791/88290716.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source_result["Match"] = result


In [161]:
df_source_result = df_source_result[["Dyad","Session","Time_begin","P1","P2","SD","QE","SV","PR","HD","Match"]]
df_source_result.to_csv("fuzzy_test_class_only.csv")

In [134]:
import pandas as pd
df_fusion = pd.read_csv("fuzzy_test_hd.csv", index_col = [0])

In [19]:
for label in ["SD", "QE", "PR", "SV", "HD"]:
    n_result = len(df_result[(df_result[label] == 'x') & (df_result.Dyad.isin(dyad_list))])
    n_source = len(df_source[(df_source[label] == 'x') & (df_source.Dyad.isin(dyad_list))])
    print(f"Number of {label} in result df : {n_result}")
    print(f"Number of {label} in source df : {n_source}")

Number of SD in result df : 336
Number of SD in source df : 432
Number of QE in result df : 78
Number of QE in source df : 88
Number of PR in result df : 79
Number of PR in source df : 63
Number of SV in result df : 2089
Number of SV in source df : 1342
Number of HD in result df : 356
Number of HD in source df : 426


In [227]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# compare_P1 = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()
# compare_P2 = pd.MultiIndex.from_product([df_source['P2'], df_target['P2']]).to_series()
# compare_D = pd.MultiIndex.from_product([df_source['D'], df_target['D']]).to_series()
# compare_S = pd.MultiIndex.from_product([df_source['S'], df_target['S']]).to_series()
# compare_S = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()
# compare_S = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()

# comparisons = [compare_P1, compare_P2, compare_D, compare_S]

# def metrics(tup):
#     return pd.Series([fuzz.ratio(*tup),
#                       fuzz.token_sort_ratio(*tup)],
#                      ['ratio', 'token'])``

# for comp_ in comparisons:
#     comp_.apply(metrics)
#     comp_.apply(metrics).unstack().idxmax().unstack(0)
#     comp_.apply(metrics).unstack(0).idxmax().unstack(0)

SyntaxError: invalid syntax (721831928.py, line 16)